# Mark II

In [17]:
!pip install parmap

In [1]:
import pandas as pd
import numpy as np
import os 
from Bio import SeqIO
import csv
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

from multiprocessing import Pool

num_partitions = 18 #number of partitions to split dataframe
num_cores = 20 #number of cores on your machine


## Create Bad list

In [4]:
# construct badlist
amp_uniref_df = pd.read_csv('/mnt/vdb/thesis/uniref50/badlist.ready.nr', sep='\t', header=0)
amp_uniref_df

,ID_Query,ID_Target,sequence_identity,alignment_length,number_of_mismatches,number_of_gap_openings,start-position_in_query,end-position_in_query,start-position_in_target,end-position_in_target,E-value,bit_score
0,UPI0004FCCC5E,UniRef50_K7EXB5,0.984,129,2,0,1,129,1,129,6.466000e-73,246
1,UPI0004FCCC5E,UniRef50_A0A093G9N1,0.961,129,5,0,1,129,1,129,5.383000e-71,241
2,UPI0004FCCC5E,UniRef50_Q16777,0.953,129,6,0,1,129,1,129,1.012000e-70,240
3,UPI0004FCCC5E,UniRef50_Q8IUE6,0.984,125,2,0,1,125,1,125,1.389000e-70,240
4,UPI0004FCCC5E,UniRef50_L7MTL8,0.945,129,7,0,1,129,1,129,2.612000e-70,239
...,...,...,...,...,...,...,...,...,...,...,...,...
7175653,MGYP001013544865,UniRef50_D8LWM1,0.630,100,36,0,22,118,15,114,5.172000e-30,123
7175654,MGYP001013544865,UniRef50_A0A6P7YQR2,0.592,108,44,0,11,118,13,120,3.445000e-29,120
7175655,MGYP001013544865,UniRef50_A0A433DDU2,0.608,118,45,0,4,118,6,123,4.726000e-29,120
7175656,MGYP001013544865,UniRef50_A0A1I8AJC7,0.533,120,55,0,1,119,1,120,3.944000e-27,114


In [5]:
bad_list = list(set(amp_uniref_df[amp_uniref_df['sequence_identity'] >=0.3]['ID_Target'].tolist())) 
len(bad_list)

125698

In [ ]:
bad_list

In [20]:
with open('/mnt/vdb/thesis/uniref50/badlist.sim30.uniq.txt', 'w') as f:
    for item in bad_list:
        f.write("%s\n" % item)

In [23]:
if "UniRef50_B4IPI1" in bad_list:
    print("True")

True


## Begin non-AMP Search

In [2]:
amp_fasta = '/mnt/vdb/thesis/amp_final/ampV3.ready.fasta'
nonAMPs_csv = "/mnt/vdb/thesis/non_amp/non_AMPs.3.csv"
uniref50= "/mnt/vdb/thesis/uniref50/uniref50.excludedBadList.len2400.fasta"

In [3]:
with open(amp_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

AMPs_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
AMPs_df # 127018

,ID,Sequence,length
0,EN92515250|C|B3FJD7|phage,MAKKSVPLRKPAGSDGQGNIKVPGGPVVLDLGDFDDIFGPMESESP...,2337
1,EN54061055|C|F8SJ56|phage,MASKKTTLPKPKGINPQGSIVQLDLDDFDDLFDEDFGLPKKNSPYT...,2319
2,EN4815120|C|Q8SCY1|phage,MAKKVTLPKGQTGATGTTLGQAGNILDLSDVDDIFGDTPKAKKGSP...,2237
3,EN3016141|D|D2J8A7|bacteriocin,MAETIKGLRIDLSLKDMGVGRSITELKRSFRTLNSDLKVSSKNFEY...,1619
4,EN9175723|BD|Q93IM3|bacteriocin,MAKKKNTYKVPSIIALTLAGTALTTHHAQAADKTQDQSTNKNILND...,1564
...,...,...,...
127013,MGYP001502946729,MKVMASVKKICRNCKVIKRRGVVRVICTDPRHKQRQG,37
127014,MGYP001502959890,MAHKKGASSSRNGRDSNAKRLGVKRFGGQLVNAGEILVRQRGTHFH...,84
127015,MGYP001502962667,MELVELLFRAEKRLSGLHPLVADKARALIRKAYEEGIYIIITQGFR...,156
127016,MGYP000753212479,MPRSLKKGPFVDHHLLSKVLSAVEANLKKPIKTWSRRSMIIPDMIG...,91


In [4]:
df_count=AMPs_df.groupby('length').length.count().reset_index(name='count').sort_values(['count'], ascending=False)
#df_count.to_csv('df_count.csv', index=False)
_tmp_df_count =df_count
_tmp_df_count

,length,count
26,37,2132
81,92,1880
82,93,1262
73,84,1259
67,78,1245
...,...,...
1356,1456,1
1352,1452,1
1351,1451,1
1345,1444,1


In [6]:
# shuffle ID
_tmp_df_count = _tmp_df_count.sample(frac=1).reset_index(drop=True)
_tmp_df_count["count"].values

array([ 4,  2, 54, ...,  1, 15,  1])

In [7]:

num_partitions = 24 #number of partitions to split dataframe 22
num_cores = 24 #number of cores on your machine 22

In [ ]:
"""
def parallelize_dataframe(df, func):
    _tmp_df_split = np.array_split(df, num_partitions)
    non_amp_data_df = pd.DataFrame(columns=['ID','Sequence','length']) 

    _list =[ [non_amp_data_df , _tmp_df_split] ]
    
    pool = Pool(num_cores)
    _list = pool.map(func,non_amp_data_df,_tmp_df_split)
    #non_amp_data_df ,_tmp_df_count = _list[0], _list[1]
    
   # non_amp_data_df = pd.concat(non_amp_data_df)
   # _tmp_df_count = pd.concat(_tmp_df_count)
    pool.close()
    pool.join()
    return _tmp_df_count, non_amp_data_df

def find_amp(_tmp_non_amp_data_df, _tmp_df_split):
    #_tmp_non_amp_data_df, _tmp_df_count = _list[0], _list[1]
    #_tmp_df_count = pd.concat(_tmp_df_count)
    #print(type(_tmp_non_amp_data_df))
    #print("-------------------")
    #print(_tmp_df_count)
    #NonAMPs_csv = open(nonAMPs_csv, 'w')
    #NonAMPs_csv.write('"ID","Sequence","length"'+ '\n')
    record = SeqIO.parse(uniref50, "fasta")
    for seq_record in record:

        seqLen = len(str(seq_record.seq))
        # update value
        _tmp_row = _tmp_df_count.loc[ _tmp_df_count['length'] == seqLen]
        if not _tmp_row.empty and  _tmp_row['count'].values > 0:
            print (str(_tmp_row['length']) +" "+ str(_tmp_row['count']))
          #  print('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"')
           # NonAMPs_csv.write('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"' + '\n')
            _tmp_non_amp_data_df.append({'ID': str(seq_record.id), 'Sequence': str(seq_record.seq)
                                     , 'length': str(seqLen)},ignore_index=True)
            _tmp_df_count.at[ _tmp_row.index,'count']= _tmp_row['count'] - 1
        #elif : # shulff seqeunce and subseq
        else:
            if _tmp_df_count['count'].values.sum() == 0:
                print("stop: found every range")
                break
    #NonAMPs_csv.close()
    _list =   [_tmp_non_amp_data_df, _tmp_df_count] 
    print(_list)
    return _list
"""

In [8]:
from os import getpid

def find_amp(_tmp_df_count):
    process_id =str(getpid())
    nonAMPs_csv = "/mnt/vdb/thesis/non_amp/non_amp."+process_id+".csv"
    NonAMPs_csv = open(nonAMPs_csv, 'w')
    NonAMPs_csv.write('"ID","Sequence","length"'+ '\n')
    print(process_id+" Start")
    record = SeqIO.parse(uniref50, "fasta")
    for seq_record in record:

        seqLen = len(str(seq_record.seq))
        # update value
        _tmp_row = _tmp_df_count.loc[ _tmp_df_count['length'] == seqLen]
        if not _tmp_row.empty and  _tmp_row['count'].values > 0:
           # print (str(_tmp_row['length']) +" "+ str(_tmp_row['count']))
           # print('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"')
            NonAMPs_csv.write('"'+str(seq_record.id)+'","'+str(seq_record.seq)+'","'+ str(seqLen)+'"' + '\n')
            
            _tmp_df_count.at[ _tmp_row.index,'count']= _tmp_row['count'] - 1
        #elif : # shulff seqeunce and subseq
                    
        else:
            if _tmp_df_count['count'].values.sum() == 0:
                print(process_id+" stop: found every range")
                break
    NonAMPs_csv.close()


from functools import partial

def parallelize_dataframe(df, func):
    _tmp_df_split = np.array_split(df, num_partitions)
        
    pool = Pool(num_cores)
    pool.map(func, _tmp_df_split)

    pool.close()
    pool.join()


In [9]:
parallelize_dataframe(_tmp_df_count, find_amp)

30506 Start30507 Start30508 Start
30512 Start
30522 Start30519 Start
30525 Start30521 Start
30524 Start30513 Start30509 Start30511 Start30523 Start30514 Start
30520 Start30510 Start30528 Start30529 Start30527 Start






30526 Start






30515 Start30516 Start


30517 Start30518 Start

30519 stop: found every range
30521 stop: found every range
30520 stop: found every range
30510 stop: found every range
30528 stop: found every range
30524 stop: found every range
30515 stop: found every range
30513 stop: found every range
30526 stop: found every range
30525 stop: found every range
30514 stop: found every range
30527 stop: found every range
30506 stop: found every range
30518 stop: found every range
30523 stop: found every range
30529 stop: found every range
30517 stop: found every range
30522 stop: found every range
30516 stop: found every range
30511 stop: found every range
30508 stop: found every range
30509 stop: found every range
30507 stop: found every range
30512 stop: found ever

## Merge non-AMPs and Check 

In [10]:
import glob

path = r'/mnt/vdb/thesis/non_amp/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

nonAMP_df = pd.concat(li, axis=0, ignore_index=True)

In [11]:
nonAMP_df

,ID,Sequence,length
0,UniRef50_A7RGL2,EESEDEDKDVVLKSDAITKYYTIKDELGKGRFGVVCKCVNKKTGKQ...,311
1,UniRef50_S4NVH8,PSNPPVFTKKMQPCRVFEHEQARFEVEFDGDPLPTIKWYRENFPIK...,75
2,UniRef50_A0A0S7JAH0,MQLSGGNPRLNQLCARWQQVWLLALDRQRKLNDALDRQEELKEFAN...,282
3,UniRef50_A0A1V3ZYL8,HAGIDPTTLHGTDTAVFTGVMAQEYGPRLYEPSQGTDGYLLTGNTS...,374
4,UniRef50_F6V062,MVDTPEMVRARENQSFLSQVNYKSDVESMKGRNHSTILDTPELRQV...,374
...,...,...,...
127013,UniRef50_U4KZ92,MHLEFNRARAEYQSTPCATKPFLWL,25
127014,UniRef50_F6KAZ0,MQFFIFVSILCLFLENVGAFYMYSS,25
127015,UniRef50_A0A2S4WHR4,MGNMFLIRPMEASHSSASGVPSLCG,25
127016,UniRef50_A0A0G4NLI8,HHALAPLPRNRHHVARRPRRRRPPR,25


In [12]:
nonAMP_df.drop_duplicates(subset=['Sequence'],inplace=True)
nonAMP_df

,ID,Sequence,length
0,UniRef50_A7RGL2,EESEDEDKDVVLKSDAITKYYTIKDELGKGRFGVVCKCVNKKTGKQ...,311
1,UniRef50_S4NVH8,PSNPPVFTKKMQPCRVFEHEQARFEVEFDGDPLPTIKWYRENFPIK...,75
2,UniRef50_A0A0S7JAH0,MQLSGGNPRLNQLCARWQQVWLLALDRQRKLNDALDRQEELKEFAN...,282
3,UniRef50_A0A1V3ZYL8,HAGIDPTTLHGTDTAVFTGVMAQEYGPRLYEPSQGTDGYLLTGNTS...,374
4,UniRef50_F6V062,MVDTPEMVRARENQSFLSQVNYKSDVESMKGRNHSTILDTPELRQV...,374
...,...,...,...
127013,UniRef50_U4KZ92,MHLEFNRARAEYQSTPCATKPFLWL,25
127014,UniRef50_F6KAZ0,MQFFIFVSILCLFLENVGAFYMYSS,25
127015,UniRef50_A0A2S4WHR4,MGNMFLIRPMEASHSSASGVPSLCG,25
127016,UniRef50_A0A0G4NLI8,HHALAPLPRNRHHVARRPRRRRPPR,25


In [13]:
nonAMP_df.length.describe()

count    127018.000000
mean        238.278929
std         300.371445
min          11.000000
25%          63.000000
50%          98.000000
75%         298.000000
max        2337.000000
Name: length, dtype: float64

In [14]:
AMPs_df.length.describe()

count    127018.000000
mean        238.278929
std         300.371445
min          11.000000
25%          63.000000
50%          98.000000
75%         298.000000
max        2337.000000
Name: length, dtype: float64

In [ ]:
### non AMP to FASTA

with open("/mnt/vdb/thesis/non_amp/NonAMPs.final.ready.fasta", 'w') as file:
    for index, row in df.iterrows():
        header = row['ID']
        seq = row['Sequence']
        # print(header)
        file.write('>'+ header + '\n')
        file.write(seq + '\n')
           
print("--- End of Writing ----")

## Labeling 

In [15]:
# setup
nonAMP_df['class']=1
AMPs_df['class']=0

In [16]:
ready_df = pd.concat([AMPs_df,nonAMP_df], axis=0)
ready_df

,ID,Sequence,length,class
0,EN92515250|C|B3FJD7|phage,MAKKSVPLRKPAGSDGQGNIKVPGGPVVLDLGDFDDIFGPMESESP...,2337,0
1,EN54061055|C|F8SJ56|phage,MASKKTTLPKPKGINPQGSIVQLDLDDFDDLFDEDFGLPKKNSPYT...,2319,0
2,EN4815120|C|Q8SCY1|phage,MAKKVTLPKGQTGATGTTLGQAGNILDLSDVDDIFGDTPKAKKGSP...,2237,0
3,EN3016141|D|D2J8A7|bacteriocin,MAETIKGLRIDLSLKDMGVGRSITELKRSFRTLNSDLKVSSKNFEY...,1619,0
4,EN9175723|BD|Q93IM3|bacteriocin,MAKKKNTYKVPSIIALTLAGTALTTHHAQAADKTQDQSTNKNILND...,1564,0
...,...,...,...,...
127013,UniRef50_U4KZ92,MHLEFNRARAEYQSTPCATKPFLWL,25,1
127014,UniRef50_F6KAZ0,MQFFIFVSILCLFLENVGAFYMYSS,25,1
127015,UniRef50_A0A2S4WHR4,MGNMFLIRPMEASHSSASGVPSLCG,25,1
127016,UniRef50_A0A0G4NLI8,HHALAPLPRNRHHVARRPRRRRPPR,25,1


In [22]:
dup_def = ready_df[ready_df.duplicated(['ID','Sequence'])]

In [23]:
dup_def.groupby(by='class').agg('count')

,ID,Sequence,length
class,,,


In [ ]:
ready_df.drop_duplicates(subset=['Sequence'],inplace=True)
ready_df

In [20]:
ready_df.to_csv("/mnt/vdb/thesis/AMP_NonAMPs.ready.csv",sep=",", quotechar='"',index=False, quoting=csv.QUOTE_ALL)

In [21]:
ready_df.to_csv("../datasets/thesis/AMP_NonAMPs.ready.csv",sep=",", quotechar='"',index=False, quoting=csv.QUOTE_ALL)